In [1]:
import keras, os, shutil

Using TensorFlow backend.


In [19]:
original_dataset_dir = '/Users/kjprice/.kaggle/competitions/dogs-vs-cats/'
base_dir = 'data/dogs-cats-small'

try:
    os.mkdir(base_dir)
except:
    print ('directory already exists')

directory already exists


### Move data over

In [22]:

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')
# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')
# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')
# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
# Directory with our validation cat pictures
test_cats_dir = os.path.join(test_dir, 'cats')
# Directory with our validation dog pictures
test_dogs_dir = os.path.join(test_dir, 'dogs')

try:
    os.mkdir(train_dir)
    os.mkdir(validation_dir)
    os.mkdir(test_dir)
    os.mkdir(train_cats_dir)
    os.mkdir(train_dogs_dir)
    os.mkdir(validation_cats_dir)
    os.mkdir(validation_dogs_dir)
    os.mkdir(test_cats_dir)
    os.mkdir(test_dogs_dir)
except:
    print('training, validation and test directories already exist')
    

try:
    # Copy first 1000 cat images to train_cats_dir
    fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
    for fname in fnames:
        src = os.path.join(original_dataset_dir, 'train', fname)
        dst = os.path.join(train_cats_dir, fname)
        shutil.copyfile(src, dst)

    # Copy next 500 cat images to validation_cats_dir
    fnames = ['cat.{}.jpg'.format(i) for i in range(1000, 1500)]
    for fname in fnames:
        src = os.path.join(original_dataset_dir, 'train', fname)
        dst = os.path.join(validation_cats_dir, fname)
        shutil.copyfile(src, dst)

    # Copy next 500 cat images to test_cats_dir
    fnames = ['cat.{}.jpg'.format(i) for i in range(1500, 2000)]
    for fname in fnames:
        src = os.path.join(original_dataset_dir, 'train', fname)
        dst = os.path.join(test_cats_dir, fname)
        shutil.copyfile(src, dst)

    # Copy first 1000 dog images to train_dogs_dir
    fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
    for fname in fnames:
        src = os.path.join(original_dataset_dir, 'train', fname)
        dst = os.path.join(train_dogs_dir, fname)
        shutil.copyfile(src, dst)

    # Copy next 500 dog images to validation_dogs_dir
    fnames = ['dog.{}.jpg'.format(i) for i in range(1000, 1500)]
    for fname in fnames:
        src = os.path.join(original_dataset_dir, 'train', fname)
        dst = os.path.join(validation_dogs_dir, fname)
        shutil.copyfile(src, dst)

    # Copy next 500 dog images to test_dogs_dir
    fnames = ['dog.{}.jpg'.format(i) for i in range(1500, 2000)]
    for fname in fnames:
        src = os.path.join(original_dataset_dir, 'train', fname)
        dst = os.path.join(test_dogs_dir, fname)
        shutil.copyfile(src, dst)

    print('Copied images over')
except:
    print('files already copied over')

training, validation and test directories already exist
files already copied over


## Create Model

In [4]:
from keras import layers
from keras import models

In [5]:
model = models.Sequential()

In [7]:
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)))

In [8]:
model.add(layers.MaxPooling2D((2,2)))

In [9]:
model.add(layers.Conv2D(64, (3,3), activation='relu'))

In [10]:
model.add(layers.MaxPooling2D((2,2)))

In [11]:
model.add(layers.Conv2D(128, (3,3), activation='relu'))

In [12]:
model.add(layers.MaxPooling2D((2,2)))

In [13]:
model.add(layers.Conv2D(128, (3,3), activation='relu'))

In [14]:
model.add(layers.MaxPooling2D((2,2)))

In [15]:
model.add(layers.Flatten())

In [16]:
model.add(layers.Dropout(0.5))

In [17]:
model.add(layers.Dense(512, activation='relu'))

In [18]:
model.add(layers.Dense(1, activation='sigmoid'))

In [23]:
from keras import optimizers

In [25]:
model.compile(optimizer=optimizers.RMSprop(lr=0.0004), loss='binary_crossentropy', metrics=['acc'])

### Create data augmentation and generators

In [29]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255, # prefer small numbers
    rotation_range=40, # degrees 0 - 180
    width_shift_range=0.2, # fraction of total width to translate image randomly to the left and right
    height_shift_range=0.2, # fraction of total height to translate image randomly to the up and down
    shear_range=0.2, # randomly shear
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest' # how are empty pixels (created by transformations) filled?
)

In [30]:
test_datagen = ImageDataGenerator(rescale=1./255) # do not augment test data

In [31]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150), # arbitrary resize
    batch_size=32,
    class_mode='binary'
)

Found 2000 images belonging to 2 classes.


In [32]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    batch_size=32,
    target_size=(150, 150),
    class_mode='binary'
)

Found 1000 images belonging to 2 classes.


### Here goes nothing

In [35]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=100,
    validation_data=test_generator,
    validation_steps=50
)

Epoch 1/100
100/100 [==============================] - 118s 1s/step - loss: 0.7100 - acc: 0.5134 - val_loss: 0.6900 - val_acc: 0.4962

KeyboardInterrupt: 